In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import calitp_data_analysis.magics

import geopandas as gpd
import pandas as pd

from great_tables import GT, loc, style
from slugify import slugify

import _report_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS

In [ ]:
# Comment out and leave this cell right below pandas
#district = "07 - Los Angeles"

In [ ]:
DISTRICT_DIGEST_URL = (
    "https://gtfs-digest--cal-itp-data-analyses.netlify.app/district_"
    f"{slugify(district)}"
)

In [ ]:
%%capture_parameters
district, DISTRICT_DIGEST_URL

In [ ]:
# Read in all datasets here with GTFS_DATA_DICT 
# Reran merge_operate_data.py to test this
OPERATOR_FILE = GTFS_DATA_DICT.digest_tables.operator_profiles
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

operator_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_FILE}.parquet",
    filters = [[("caltrans_district", "==", district)]]
)

# using name instead of schedule_gtfs_dataset_key allows us to get 
# the last ones for LA Metro without keeping extraneous rows for LA Metro when keys changed
operator_df = operator_df.sort_values(
    ["service_date", "name"], 
    ascending=[False, True]
).drop_duplicates(
    subset=["name"]
).reset_index(drop=True)

In [ ]:
# Need this, but sometimes we'll subset columns for table
def replace_column_names(df: pd.DataFrame):
    df.columns = df.columns.map(_report_utils.replace_column_names)
    return df

# District {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

Individual transit operators have their pages at: **[{DISTRICT_DIGEST_URL}]({DISTRICT_DIGEST_URL})**

In [ ]:
def district_stats(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """
    Get district metrics by summing or taking average across 
    all operators in the district for GTFS schedule data.
    """
    sum_me = [
        f"operator_{i}" for i in [
            "n_routes", "n_trips", "n_shapes", "n_stops", 
            "n_arrivals"]
    ]
    
    df2 = (df.groupby(group_cols, 
                      observed=True, group_keys=False)
           .agg({
               "name": "nunique",
               **{c:"sum" for c in sum_me},
           })
           .reset_index()
           .rename(columns = {"name": "n_operators"})
          )
    
    # These need to be calculated again separately
    df2 = df2.assign(
        arrivals_per_stop = df2.operator_n_arrivals.divide(df2.operator_n_stops).round(2),
        trips_per_operator = df2.operator_n_trips.divide(df2.n_operators).round(2)
    )
    
    return df2

In [ ]:
district_summary = district_stats(operator_df, "caltrans_district")

In [ ]:
def transpose_me(
    df: pd.DataFrame, 
) -> pd.DataFrame:
    """
    District summary should be transposed, otherwise columns
    get shrunk and there's only 1 row.
    
    Do some wrangling here so that great tables
    can display it fairly cleanly.
    """
    # Fix this so we can see it
    subset_df = df.drop(
        columns = "caltrans_district"
    ).reset_index(drop=True)
    
    subset_df2 = subset_df.rename(
        columns = {
            **{c: f"{c.replace('operator_n_', '# ')}" for c in subset_df.columns},
            "n_operators": "# operators",
            "arrivals_per_stop": "arrivals per stop",
            "trips_per_operator": "trips per operator"
        }).T.reset_index().rename(columns = {0: "value"})
    
    return subset_df2

In [ ]:
summary_table1 = (GT(
    district_summary.drop(
        columns = ["arrivals_per_stop", "trips_per_operator"]
    ).pipe(transpose_me)
)
 .fmt_integer(columns="value")
 .cols_label(index="")
 .tab_header(title = f"District {district} GTFS summary stats")
)

summary_table2 = (GT(
    district_summary[
        ["caltrans_district", "arrivals_per_stop", "trips_per_operator"]
    ].pipe(transpose_me)
)
 .fmt_number("value", decimals=1)
 .cols_label(index="")
)

In [ ]:
display(summary_table1)
display(summary_table2)

## GTFS Stats by Operator

In [ ]:
operators_in_district = operator_df.schedule_gtfs_dataset_key.unique()

operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
    filters = [["schedule_gtfs_dataset_key", "in", operators_in_district]],
    columns = ["name", "service_date", 
               "route_combined_name", "geometry"]
).sort_values(
    ["service_date", "name", "route_combined_name"], 
    ascending=[False, True, True]
).drop_duplicates(
    subset = ["name", "route_combined_name"]
).drop(
    columns = ["service_date", "route_combined_name"]
    # drop route because after the dissolve, all operator routes are combined
    # so route would hold only the first row's value
).dissolve(by = "name").reset_index().pipe(replace_column_names)

operator_route_gdf["Transit Operator"] = operator_route_gdf["Transit Operator"].str.replace(" Schedule", "")

In [ ]:
operator_route_gdf.explore(
    "Transit Operator", 
    tiles = "CartoDB Positron",
    categorical=True,
    legend=True,
    legend_kwds = {
        "width": 200
    }
)

In [ ]:
shared_cols = ["organization_name", "name"]
exclude_cols = ["schedule_gtfs_dataset_key", "caltrans_district",
                "organization_source_record_id", "service_date", 
                "primary_uza"]

gtfs_service_cols = [
    c for c in operator_df.columns 
    if "operator_" in c]

In [ ]:
gtfs_table_df = operator_df[
    shared_cols + gtfs_service_cols
].pipe(replace_column_names)

In [ ]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns = [
            c for c in gtfs_table_df.columns if c not in 
         ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ])
    .fmt_number(
        columns = ["Operator Service Miles", "Avg Arrivals per Stop"], 
        decimals = 1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["white", "green"],
        na_color="lightgray")
    .tab_header(
         title = f"District {district}",
         subtitle = "Daily GTFS schedule statistics by operator")
    .cols_align(columns = [c for c in gtfs_table_df.columns if c not in ["Organization", "Transit Operator"]],
        align="center")
    .opt_align_table_header(align="center")
    .tab_style(
        style=style.text(size="14px"),
        locations=loc.body())
    .tab_options(
        container_width = "100%",
        table_background_color="white",
        table_body_hlines_style="none",
        table_body_vlines_style="none",
        heading_background_color="white",
        column_labels_background_color="white",
        row_group_background_color="white",
        stub_background_color="white",
        source_notes_background_color="white"
     )
    )

gtfs_table